###**1-IMPORTS AND CONFIGURATION**

In [0]:
from pyspark.sql.functions import (col, lit, current_timestamp, md5, concat_ws, monotonically_increasing_id, input_file_name, to_date)
from pyspark.sql.types import *
from delta.tables import DeltaTable
import logging
from datetime import datetime
from typing import Dict, List, Optional
import os

In [0]:
## Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [0]:
# Configuration
CONFIG = {
    "catalog": "eldenringcatalog",
    "bronze_schema": "bronze",
    "source_path": "/Volumes/eldenringcatalog/bronze/eldenringvolume/",
    "checkpoint_path": "/Volumes/eldenringcatalog/bronze/checkpoints/",
    "batch_id": datetime.now().strftime("%Y%m%d_%H%M%S"),
    "csv_options": {
        "header": "true",
        "inferSchema": "true",
        "multiLine": "true",
        "escape": '"',
        "quote": '"'
    }
}

###**2-BRONZE LAYER UTILITIES**

In [0]:
class BronzeLoader:
    
    def __init__(self,spark,config:Dict):
        self.spark = spark
        self.config = config
        self.batch_id = config["batch_id"]
        self.catalog = config["catalog"]
        self.schema = config["bronze_schema"]
    
    def add_audit_columns(self,df,source_file:str):
        "Add audit columns to Bronze layer DataFrame"
        return(df
               .withColumn("ingestion_timestamp",current_timestamp())
               .withColumn("source_file",lit(source_file))
               .withColumn("batch_id",lit(self.batch_id))
               .withColumn("record_hash",md5(concat_ws("||",*[col(x) for x in df.columns])))
               .withColumn("ingestion_date",to_date(col("ingestion_timestamp")))
               )

    def clean_column_names(self, df):
            """
            Clean column names to be Delta Lake compatible
            - Replace spaces with underscores
            - Replace special characters
            - Convert to lowercase
            """
            for col_name in df.columns:
                # Replace spaces and special chars with underscores
                clean_name = (col_name
                    .replace(" ", "_")
                    .replace("-", "_")
                    .replace("(", "")
                    .replace(")", "")
                    .replace(",", "")
                    .replace(";", "")
                    .replace("{", "")
                    .replace("}", "")
                    .replace("\n", "_")
                    .replace("\t", "_")
                    .replace("=", "_")
                    .lower())
                
                # Rename column if it changed
                if clean_name != col_name:
                    df = df.withColumnRenamed(col_name, clean_name)
                    logger.info(f"  Renamed column: '{col_name}' → '{clean_name}'")
            
            return df



    def load_csv_to_bronze(self,csv_filename:str,table_name:str,schema:Optional[StructType]=None)->bool :
        "Load a single CSV file to Bronze Delta table"
        try:
            source_path =f"{self.config['source_path']}{csv_filename}"
            full_table_name = f"{self.catalog}.{self.schema}.{table_name}"

            logger.info(f"Loading {csv_filename} to {full_table_name}")
            
            # Read CSV              
            df = (spark.read
                    .options(**self.config["csv_options"])
                    .csv(source_path))
            
            df = self.clean_column_names(df)

            record_count = df.count()
            logger.info(f"  Read {record_count:,} records from {csv_filename}")
            # Add audit columns
            df_with_audit = self.add_audit_columns(df,csv_filename)

            #Write to Bronze Delta table (append more for incremental loads)
            (df_with_audit
             .write
             .format("delta")
             .mode("append")
             .partitionBy("ingestion_date")
             .option("mergeSchema","true")
             .saveAsTable(full_table_name))
            
            #set data 
            spark.sql(f"""
                ALTER TABLE {full_table_name} SET TBLPROPERTIES (
                    'delta.autoOptimize.optimizeWrite' = 'true',
                    'delta.autoOptimize.autoCompact' = 'true',
                    'delta.enableChangeDataFeed' = 'true'
                )
            """)
            
            logger.info(f"  ✅ Successfully loaded {record_count:,} records to {full_table_name}")
            return True
        except Exception as e:
            logger.error(f"  Failed to load {csv_filename}: {str(e)}")
            return False
    
    def check_and_create_schema(self):
        """Create schema if it doesn't exist"""
        full_schema_name = f"{self.catalog}.{self.schema}"

        spark.sql(f"""
            CREATE SCHEMA IF NOT EXISTS {full_schema_name}
            COMMENT 'Bronze layer - Raw data landing zone for Elden Ring API data'
        """)

        logger.info(f"Schema {full_schema_name} ready")

    
    def optimize_table(self,table_path:str):
        try:
            full_table_name = f"{self.catalog}.{self.schema}.{table_name}"
            
            # Optimize (compact small files)
            spark.sql(f"OPTIMIZE {full_table_name}")
            logger.info(f"  Optimized {full_table_name}")

            # Vacuum (remove old files, retain 7 days)
            spark.sql(f"VACUUM {full_table_name} RETAIN 168 HOURS")
            logger.info(f"  Vacuumed {full_table_name}")
        except Exception as e:
            logger.warning(f"  Optimization warning for {table_path}: {str(e)}")


In [0]:
%sql
use catalog `eldenringcatalog`; select * from `bronze`.`armors` limit 100;

## **3-CSV FILE SCHEMAS (Explicit for validation)**

In [0]:
# # Schema definitions
# WEAPONS_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("weapon_id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("weight", StringType(), True),
#     StructField("description", StringType(), True),
#     StructField("dlc", StringType(), True),
#     StructField("requirements", StringType(), True), # JSON
#     StructField("damage type", StringType(), True),
#     StructField("category", StringType(), True),
#     StructField("passive effect", StringType(), True),  
#     StructField("skill", StringType(), True),
#     StructField("FP cost", StringType(), True)
# ])

# SHIELDS_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("shieled_id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("weight", StringType(), True),
#     StructField("description", StringType(), True),
#     StructField("dlc", StringType(), True),  
#     StructField("requirements", StringType(), True), # JSON
#     StructField("damage type", StringType(), True),
#     StructField("category", StringType(), True),  
#     StructField("passive effect", StringType(), True),  # JSON
#     StructField("skill", StringType(), True),
#     StructField("FP cost", StringType(), True)
# ])

# LOCATIONS_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("region", StringType(), True),
#     StructField("items", StringType(), True),  # JSON array - 
#     StructField("npcs", StringType(), True),  # JSON array - 
#     StructField("creatures", StringType(), True),  # JSON array - 
#     StructField("bosses", StringType(), True) , # JSON array - 
#     StructField("description", StringType(), True),
#     StructField("dlc", StringType(), True)
# ])

# BOSSES_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("HP", StringType(), True),
#     StructField("locations & Drops", StringType(), True),  # Complex JSON
#     StructField("blockquote", StringType(), True),
#     StructField("dlc", StringType(), True)
# ])

# NPCS_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("location", StringType(), True),  # Free text
#     StructField("role", StringType(), True),
#     StructField("voiced by", StringType(), True),
#     StructField("description", StringType(), True),
#     StructField("dlc", StringType(), True)

# ])

# ARMORS_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("description", StringType(), True),
#     StructField("type", StringType(), True),
#     StructField("damage negation", StringType(), True), # JSON
#     StructField("resistance", StringType(), True),  # JSON
#     StructField("weight", StringType(), True),
#     StructField("special effect", StringType(), True),
#     StructField("how to acquire", StringType(), True), 
#     StructField("in-game section", StringType(), True),   
#     StructField("dlc", StringType(), True) 
    
# ])

# SKILLS_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("type", StringType(), True),
#     StructField("equipament", StringType(), True),
#     StructField("charge", StringType(), True),
#     StructField("FP", StringType(), True),
#     StructField("effect", StringType(), True),
#     StructField("locations", StringType(), True),
#     StructField("dlc", StringType(), True) 
    
# ])

# TALISMANS_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("effect", StringType(), True),
#     StructField("weight", StringType(), True),
#     StructField("value", StringType(), True),
#     StructField("description", StringType(), True),
#     StructField("dlc", StringType(), True) ,
#     StructField("image", StringType(), True)
        
# ])

# SORCERIES_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("description", StringType(), True),
#     StructField("effect", StringType(), True),
#     StructField("FP", StringType(), True),
#     StructField("slot", StringType(), True),
#     StructField("INT", StringType(), True),
#     StructField("FAI", StringType(), True),  
#     StructField("ARC", StringType(), True) , 
#     StructField("stamina cost", StringType(), True) ,
#     StructField("bonus", StringType(), True) ,
#     StructField("location", StringType(), True) , #free text
#     StructField("dlc", StringType(), True)
# ])

# INCANTATIONS_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("description", StringType(), True),
#     StructField("effect", StringType(), True),
#     StructField("FP", StringType(), True),
#     StructField("slot", StringType(), True),
#     StructField("INT", StringType(), True),
#     StructField("FAI", StringType(), True),  
#     StructField("ARC", StringType(), True) ,
#     StructField("stamina cost", StringType(), True) ,
#     StructField("bonus", StringType(), True),
#     StructField("group", StringType(), True),
#     StructField("location", StringType(), True),  # Free text
#     StructField("dlc", StringType(), True)    
# ])

# ASHES_OF_WAR_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("affinity", StringType(), True),
#     StructField("skill", StringType(), True),
#     StructField("description", StringType(), True),
#     StructField("dlc", StringType(), True)   
# ])

# SPIRIT_ASHES_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("type", StringType(), True),
#     StructField("FP cost", StringType(), True),
#     StructField("HP cost", StringType(), True),
#     StructField("effect", StringType(), True),
#     StructField("description", StringType(), True) ,
#     StructField("dlc", StringType(), True)   
# ])

# CREATURES_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("locations", StringType(), True),#JSON Array
#     StructField("drops", StringType(), True),#JSON Array
#     StructField("blockquote", StringType(), True),
#     StructField("dlc", StringType(), True)
# ])

# WEAPONS_UPGRADE_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("weapon name", StringType(), False),
#     StructField("upgrade", StringType(), False),
#     StructField("attack power", StringType(), False), #JSON
#     StructField("stat scaling", StringType(), False), #JSON
#     StructField("passive effects", StringType(), False),#JSON
#     StructField("damage reduction (%)", StringType(), False),#JSON

# ])

# SHIELDS_UPGRADE_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("shield name", StringType(), False),
#     StructField("upgrade", StringType(), False),
#     StructField("attack power", StringType(), False), #JSON
#     StructField("stat scaling", StringType(), False), #JSON
#     StructField("passive effects", StringType(), False),#JSON
#     StructField("damage reduction (%)", StringType(), False),#JSON

# ])

# AMMOS_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("type", StringType(), True),                    # Arrow/Bolt/Greatarrow
#     StructField("damage type", StringType(), True), 
#     StructField("attack power", StringType(), True),                  # JSON OBJECT
#     StructField("passive effect", StringType(), True),
#     StructField("description", StringType(), True),
#     StructField("dlc", StringType(), True)                 
# ])

# BELLS_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("type", StringType(), True),                    # Bell Bearing type
#     StructField("effect", StringType(), True) ,                 # What items it unlocks
#     StructField("description", StringType(), True),
#     StructField("dlc", StringType(), True)     
# ])

# CONSUMABLES_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("effect", StringType(), True),                  # What it does
#     StructField("FP cost", StringType(), True),
#     StructField("description", StringType(), True),
#     StructField("dlc", StringType(), True)                    
                       
# ])

# COOKBOOKS_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("effect", StringType(), True), 
#     StructField("required for", StringType(), True) ,            # JSON array of craftable items
#     StructField("description", StringType(), True),
#     StructField("dlc", StringType(), True)
                           
# ])

# CRYSTAL_TEARS_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("effect", StringType(), True) ,
#     StructField("FP cost", StringType(), True), 
#     StructField("description", StringType(), True),
#     StructField("dlc", StringType(), True)
                     
# ])

# GREAT_RUNES_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("type", StringType(), True),
#     StructField("description", StringType(), True),
#     StructField("effect", StringType(), True),                  # Rune power
#     StructField("boss", StringType(), True) ,                    # Which boss drops it
#     StructField("locations", StringType(), True),
#     StructField("divine tower locations", StringType(), True),
#     StructField("dlc", StringType(), True)
# ])

# KEY_ITEMS_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("description", StringType(), True),
#     StructField("type", StringType(), True) ,                    # Key Item category
#     StructField("location", StringType(), True),
#     StructField("dlc", StringType(), True)
# ])

# MATERIALS_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("effect", StringType(), True),                   # Used for crafting..
#     StructField("FP cost", StringType(), True), 
#     StructField("description", StringType(), True),
#     StructField("dlc", StringType(), True)                           
# ])

# MULTI_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("type", StringType(), True),
#     StructField("effect", StringType(), True),
#     StructField("description", StringType(), True),
#     StructField("dlc", StringType(), True) 
                       
# ])

# REMEMBRANCES_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("type", StringType(), True),
#     StructField("description", StringType(), True),
#     StructField("option 1", StringType(), True),                    
#     StructField("option 2", StringType(), True),                  # What items you can trade for
#     StructField("value", StringType(), True),               # Rune value
#     StructField("boss", StringType(), True),                     # Which boss drops it
#      StructField("dlc", StringType(), True) 
# ])

# TOOLS_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("description", StringType(), True),
#     StructField("type", StringType(), True),                    
#     StructField("usage", StringType(), True),   
#     StructField("location", StringType(), True), 
#     StructField("dlc", StringType(), True)               
# ])

# UPGRADE_MATERIALS_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("description", StringType(), True),                 
#     StructField("effect", StringType(), True) ,
#     StructField("dlc", StringType(), True)

# ])

# WHETBLADES_SCHEMA = StructType([
#     StructField("id", StringType(), False),
#     StructField("name", StringType(), False),
#     StructField("image", StringType(), True),
#     StructField("description", StringType(), True),
#     StructField("usage", StringType(), True)   ,
#     StructField("location", StringType(), True),
#     StructField("dlc", StringType(), True)
# ])


In [0]:
# # Get all tables from the schema
# tables = spark.catalog.listTables("eldenringcatalog.bronze")

# for t in tables:
#     table_name = f"eldenringcatalog.bronze.{t.name}"
#     print(f"Dropping table: {table_name}")
#     spark.sql(f"DROP TABLE IF EXISTS {table_name}")

## **4-BRONZE LAYER TABLE DEFINITIONS**

In [0]:
BRONZE_TABLES = [
    # Main entity files (15)
    {"csv": "weapons.csv", "table": "weapons", "schema": None},
    {"csv": "shields.csv", "table": "shields", "schema": None},
    {"csv": "armors.csv", "table": "armors", "schema": None},
    {"csv": "bosses.csv", "table": "bosses", "schema": None},
    {"csv": "npcs.csv", "table": "npcs", "schema": None},
    {"csv": "creatures.csv", "table": "creatures", "schema": None},
    {"csv": "locations.csv", "table": "locations", "schema": None},
    {"csv": "skills.csv", "table": "skills", "schema": None},
    {"csv": "talismans.csv", "table": "talismans", "schema": None},
    {"csv": "sorceries.csv", "table": "sorceries", "schema": None},
    {"csv": "incantations.csv", "table": "incantations", "schema": None},
    {"csv": "ashesOfWar.csv", "table": "ashes_of_war", "schema": None},
    {"csv": "spiritAshes.csv", "table": "spirit_ashes", "schema": None},
    {"csv": "weapons_upgrades.csv", "table": "weapons_upgrades", "schema": None},
    {"csv": "shields_upgrades.csv", "table": "shields_upgrades", "schema": None},
    
    # Items subdirectory (13 files)
    {"csv": "ammos.csv", "table": "items_ammos", "schema": None},
    {"csv": "bells.csv", "table": "items_bells", "schema": None},
    {"csv": "consumables.csv", "table": "items_consumables", "schema": None},
    {"csv": "cookbooks.csv", "table": "items_cookbooks", "schema": None},
    {"csv": "crystalTears.csv", "table": "items_crystal_tears", "schema": None},
    {"csv": "greatRunes.csv", "table": "items_great_runes", "schema": None},
    {"csv": "keyItems.csv", "table": "items_key_items", "schema": None},
    {"csv": "materials.csv", "table": "items_materials", "schema": None},
    {"csv": "multi.csv", "table": "items_multi", "schema": None},
    {"csv": "remembrances.csv", "table": "items_remembrances", "schema": None},
    {"csv": "tools.csv", "table": "items_tools", "schema": None},
    {"csv": "upgradeMaterials.csv", "table": "items_upgrade_materials", "schema": None},
    {"csv": "whetblades.csv", "table": "items_whetblades", "schema": None}
]

## **5-MAIN INGESTION ORCHESTRATION**

In [0]:
def run_bronze_ingestion(spark,config: Dict) -> Dict:

    logger.info("="*80)
    logger.info("STARTING BRONZE LAYER INGESTION (UNITY CATALOG)")
    logger.info(f"Catalog: {config['catalog']}")
    logger.info(f"Schema: {config['bronze_schema']}")
    logger.info(f"Batch ID: {config['batch_id']}")
    logger.info("="*80)

    loader = BronzeLoader(spark,config)

    # Create bronze database
    loader.check_and_create_schema()

    # Statistics tracking
    stats = {
        "total_files": len(BRONZE_TABLES),
        "successful": 0,
        "failed": 0,
        "failed_files": [],
        "start_time": datetime.now()
    }

    # Load all CSV files
    for i, table_def in enumerate(BRONZE_TABLES, 1):
        logger.info(f"\n[{i}/{stats['total_files']}] Processing {table_def['csv']}")

        success = loader.load_csv_to_bronze(
            csv_filename=table_def["csv"],
            table_name=table_def["table"],
            schema=table_def.get("schema")
        )

        if success:
            stats["successful"] += 1
        else:
            stats["failed"] += 1
            stats["failed_files"].append(table_def["csv"])

    stats["end_time"] = datetime.now()
    stats["duration"] = (stats["end_time"] - stats["start_time"]).total_seconds()

    # Summary
    logger.info("\n" + "="*80)
    logger.info("BRONZE LAYER INGESTION COMPLETE")
    logger.info("="*80)
    logger.info(f"Total files: {stats['total_files']}")
    logger.info(f"✅ Successful: {stats['successful']}")
    logger.info(f"❌ Failed: {stats['failed']}")
    logger.info(f"⏱️ Duration: {stats['duration']:.2f} seconds")

    if stats["failed_files"]:
        logger.warning(f"Failed files: {', '.join(stats['failed_files'])}")
    
    return stats




## **6-INCREMENTAL LOAD FUNCTIONS**

In [0]:
def load_incremental_bronze(spark,
                            csv_filename: str, 
                            table_name: str,
                            config: Dict) -> int:
    
    loader = BronzeLoader(config)
    source_path = f"{config['source_path']}{csv_filename}"
    full_table_name = f"{config['catalog']}.{config['bronze_schema']}.{table_name}"

    # Read incoming data
    df_incoming = (spark.read
        .options(**config['csv_options'])
        .csv(source_path))
    
    df_incoming_with_hash = df_incoming.withColumn(
        "record_hash",
        md5(concat_ws("||", *[col(c) for c in df_incoming.columns]))
    )

    # Read existing Bronze data
    try :
        df_existing = spark.table(full_table_name)

        # Find new or changed records (hash not in existing)
        df_new_or_changed =(df_incoming_with_hash
                            .join(df_existing.select("id","record_hash").distinct(), on="record_hash", how="left_anti"))
        
        new_count = df_new_or_changed.count()
        if new_count > 0:
            #add audit columns and append
            df_to_load =loader.add_audit_columns(df_new_or_changed,csv_filename)

            (df_to_load
                .write
                .format("delta")
                .mode("append")
                .partitionBy("ingestion_date")
                .saveAsTable(full_table_nameh))
            
            logger.info(f"  ✅ Loaded {new_count:,} new/changed records to {full_table_name}")
        else:
            logger.info(f"  ℹ️ No changes detected for {full_table_name}")
        
        return new_count
    
    except Exception as e:
        # If table doesn't exist, do full load
        logger.warning(f"Table {full_table_name} doesn't exist, performing full load")
        loader.load_csv_to_bronze(csv_filename, full_table_name)
        return df_incoming.count()

## **7-DATA QUALITY CHECKS**

In [0]:
def validate_bronze_tables(spark,config: Dict) -> Dict:

    logger.info("\n" + "="*80)
    logger.info("VALIDATING BRONZE LAYER DATA QUALITY")
    logger.info("="*80)

    validation_results = {}
    catalog = config["catalog"]
    schema = config["bronze_schema"]

    for table_def in BRONZE_TABLES:
        table_name = table_def["table"]
        full_table_name = f"{catalog}.{schema}.{table_name}"
    
        try:
            df = spark.table(full_table_name)
            # Basic checks
            total_records = df.count()
            null_ids = df.filter(col("id").isNull()).count()
            duplicate_ids = df.groupBy("id").count().filter(col("count") > 1).count()

            validation_results[full_table_name] = {
                "status": "PASS" if null_ids == 0 else "FAIL",
                "total_records": total_records,
                "null_ids": null_ids,
                "duplicate_ids": duplicate_ids,
                "distinct_ids": df.select("id").distinct().count()
            }

            logger.info(f"\n{full_table_name}:")
            logger.info(f"  Total records: {total_records:,}")
            logger.info(f"  Null IDs: {null_ids}")
            logger.info(f"  Duplicate IDs: {duplicate_ids}")
            logger.info(f"  Status: {validation_results[full_table_name]['status']}")
        except Exception as e:
            logger.error(f"Validation failed for {full_table_name}: {str(e)}")
            validation_results[full_table_name] = {"status": "ERROR", "error": str(e)}

    return validation_results
            

%md
## **7-UTILITY FUNCTIONS**

In [0]:
def create_bronze_error_log_table(
    spark,
    config
):
    catalog = config["catalog"]
    schema = config["bronze_schema"]
    error_table = f"{catalog}.{schema}.bronze_error_log"
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {error_table} (
            error_timestamp TIMESTAMP NOT NULL,
            batch_id STRING NOT NULL,
            source_file STRING NOT NULL,
            table_name STRING NOT NULL,
            error_type STRING NOT NULL,
            error_message STRING,
            record_data STRING
        )
        USING DELTA
    """)
        
    logger.info(f"✅ Error log table ready: {error_table}")

def get_bronze_table_stats(spark,table_name: str, config: Dict) -> Dict:
    """Get statistics for a Bronze table"""
    catalog = config["catalog"]
    schema = config["bronze_schema"]
    full_table_name = f"{catalog}.{schema}.{table_name}"
    
    try:
        df = spark.table(full_table_name)
        
        stats = {
            "table_name": full_table_name,
            "total_records": df.count(),
            "distinct_ids": df.select("id").distinct().count(),
            "partitions": df.select("ingestion_date").distinct().count(),
            "latest_ingestion": df.agg({"ingestion_timestamp": "max"}).collect()[0][0],
            "earliest_ingestion": df.agg({"ingestion_timestamp": "min"}).collect()[0][0]
        }
        
        return stats
        
    except Exception as e:
        logger.error(f"Failed to get stats for {full_table_name}: {str(e)}")
        return {"table_name": full_table_name, "error": str(e)}

def optimize_all_bronze_tables(spark,config: Dict):
    """Optimize all Bronze tables"""
    logger.info("\n" + "="*80)
    logger.info("OPTIMIZING ALL BRONZE TABLES")
    logger.info("="*80)
    
    loader = BronzeLoader(spark, config)
    
    for table_def in BRONZE_TABLES:
        table_name = table_def["table"]
        logger.info(f"\nOptimizing {table_name}...")
        loader.optimize_table(table_name)
    
    logger.info("\n✅ All tables optimized")

## **7-MAIN EXECUTION**

In [0]:

create_bronze_error_log_table(spark,CONFIG)
        
        # Run full Bronze ingestion
ingestion_stats = run_bronze_ingestion(spark,CONFIG)
        
        # Validate data quality
validation_results = validate_bronze_tables(spark,CONFIG)
        
        # Print summary statistics
for table_def in BRONZE_TABLES[:5]:
    stats = get_bronze_table_stats(
        spark,
        table_def["table"],
        CONFIG
    )
    total_records = stats.get('total_records')
    distinct_ids = stats.get('distinct_ids')
    latest_ingestion = stats.get('latest_ingestion', 'N/A')

    logger.info(f"\n{stats['table_name']}:")
    logger.info(f"  Records: {total_records:,}" if isinstance(total_records, int) else f"  Records: {total_records}")
    logger.info(f"  Distinct IDs: {distinct_ids:,}" if isinstance(distinct_ids, int) else f"  Distinct IDs: {distinct_ids}")
    logger.info(f"  Latest load: {latest_ingestion}")       
        
        
logger.info("\n✅ Bronze layer ingestion pipeline completed successfully")
        
   

In [0]:
# tables = spark.sql("SHOW TABLES IN eldenringcatalog.bronze").collect()

# # Create a list to store results
# results = []

# for table in tables:
#     table_name = table.tableName
    
#     # Count rows in each table
#     try:
#         count_df = spark.sql(f"SELECT COUNT(*) as count FROM eldenringcatalog.bronze.{table_name}")
#         row_count = count_df.collect()[0]['count']
#         results.append((table_name, row_count))
#         print(f"✅ {table_name}: {row_count:,} rows")
#     except Exception as e:
#         print(f"❌ Error counting {table_name}: {str(e)}")
#         results.append((table_name, 0))

# # Create a DataFrame with the results
# from pyspark.sql.types import StructType, StructField, StringType, LongType

# schema = StructType([
#     StructField("table_name", StringType(), False),
#     StructField("row_count", LongType(), False)
# ])

# results_df = spark.createDataFrame(results, schema)
# results_df = results_df.orderBy("table_name")